# Lecture 3
Monitoring solution process is important. While looking at numbers stored in a text file is fun, it is much easier to comprehend data using graphs and plots.

We will:
* Look at a (possibly) better way than lists to handle sequences of numerical data - NumPy and arrays
* Look at reading numerical data from files (stored as columns of numbers, but other formats can also be read)
* Use the MatPlotLib library to produce some simple line, bar and color plots.

* Have a look at SciPy to illustrate capabilities, find some function extrema and roots, perform some interpolation (and extrapolation).

## NumPy and arrays
We will now work with NumPy and SciPy and introduce selected features that might be useful to us. By no means we wil cover all features, and when in need please consult appropriate (and very rich) online documentation.

### NumPy
NumPy is a numerical method library that implements a number of functionalities. The most important to us is going to be the *array*. While using *list* is fine there is a cost associated with the fact that list can hold anything (actually mixing types).
This has a consequence that any operation needs first to verify the type, making lists not so efficient, especially for larger problems.
On the other hand NumPy arrays store data of only the declared type and have a constant size. Also much of NumPy functions are calls to compiled binary code (we will be writing such code!).  

Let's start by importing arrays from NumPy:

In [ ]:
import numpy as np

We will refer to elements of NumPy using *np* (less writing). Our first array will be a 1D vector of integers:

In [ ]:
a = np.array([1,2,3,4])
print(a)

**Coding time!**: Experiment with arrays. Make and print some. Try passing lists in lists.

### Creating arrays
There is a number of ways arrays can be created with numpy.array() function. The easiest is to use an existing list, or use the advanced list initialization with \[\]-brackets. There is also a number of special *numpy* functions to create arrays. Let's see.

* Array from list:

In [ ]:
l = list(range(0,10))
print(l)
a = np.array(l)
print(a)

In [ ]:
a = np.array([x for x in l if x%2])
print(a)
a = np.array([ 2*x+3 for x in l ])
print(a)

**Coding time!**: Create some arrays using the advanced syntax as above. E.g. use `range` to create an array of data from -10 to 10 and another one that is the square of the first one.

fun, right?
* From other array, by simple arithmetic operation:

In [ ]:
b = a**2 #!
print(b)
b = a+a
print(b)
b = 2*a + 8
print(b)

**Coding time!**: Modify the previous example using the `**` operator directly on an array. 

* Using *numpy.array* functions (see docs)
    * empty - an uninitialized array,
    * ones - an array with values set to one,
    * zeros - an array with values set to zeros,
    * full - an array filled with value.
    * \*_like - array with the shape and type of input

In [ ]:
a = np.ones(5)
print(a)
a = np.zeros(5)
print(a)
a = np.full(5, 10)
print(a)
a = np.zeros_like(a)
print(a)

For now, we did not specify the type stored by the array. Rather the type was determined for us based on our input. Look closely at the elements of arrays above. What is their type?  
**dtype** member of array stores this information.

In [ ]:
a = np.ones(5)
print(a.dtype)
a = np.full(5, 10)
print(a.dtype) # why the difference?

We can specify the desired type at creation:

In [ ]:
a = np.array(range(0,10))
print(a.dtype)
a = np.array(range(0,10), dtype=float)
print(a.dtype)

* Numpy `arrange` and `linspace`  
`arrange`:  regularly incrementing values  
`linspace`: array with a specified number of elements

In [ ]:
np.arange(5)

In [ ]:
np.arange(-1, 1, 0.5)

In [ ]:
np.linspace(-1., 1., 11)

* random values  
NumPy provides a a random number generator to create random values from 0 to 1. It is a part of `random` module via e.g. `default_rng(seed)` generator. Like this:

In [ ]:
np.random.default_rng(10).random(5)

### Multidimensional arrays
NumPy allows for creation of multidimensional arrays. Personally I try not to use those to often, assuming that if you have a problem complex enough to need a matrix you would be probably better of using C or C++ for it. But that is an opinion and a biased one at that!  
NumPy arrays have `shape`s, that defines how values can be accessed.  
Lets see:

In [ ]:
a = np.ones(5)
print(a)
print(a.shape)
print(type(a.shape))

So `shape` is an attribute of an object, and a tuple at that - we recall it is **immutable**, meaning we can read but we can not modify the entries. Upon creation we can set the shape, by passing a tuple.

In [ ]:
a = np.ones((2,2))
print(a)

and access elements like this:

In [ ]:
a[0,0] = 0
a[0,1] = 1
a[1,0] = 10
a[1,1] = 11
print(a[0])
print(a[1:2])
print(a[1:2][0])

Another (possibly useful) atribute is size:

In [ ]:
a[0].size

#### Special arrays:
`eye` - identity matrix

In [ ]:
np.eye(5)

`diag` - exactly what you think:

In [ ]:
np.diag(np.linspace(-1,1,5))

There is much more than is possible with arrays creation and modification. For now let us just try to use arrays for some simple operations:  
* `trnaspose()`
* `dot()`

In [ ]:
x = np.linspace(-1,1,3)
x = np.transpose(x)
A = np.array([[1,1,0],[0,2,0],[0,0,2]])
print(A)
print(A.dot(x))

And some fun with arrays, that might come in handy (or not), that is solving an eigen problem:

In [ ]:
from numpy import linalg as LA
sig, v = LA.eig(A)
print(sig)
print(v)

## Processing files - IO operations on files
We will now perform simple read and write operations on files. Our ultimate goal is to process files which contain settings for software packages we use. For example in an optimization loop, we might need to manipulate files defining parameters for the simulator and computed by the optimizer. Similarly we will need the value of the objective function to be read and passed to the optimizer.

We will start just with opening, writing and reading. Simple stuff:

In [ ]:
f = open('burza', 'r') # similarly to printf there is a flag that specifies what to do: 'w', 'r', 'a'
a = f.readline()
print(a, end='')
a = f.readline()
print(a, end='')
f.close()

In [ ]:
f = open('burza', 'r')
alllines = f.readlines()
for line in alllines:
    print(line)
f.close()

In [ ]:
f = open('burza', "r")
g = open('burza2', "w")

for line in f:
    g.write(line)
f.close()
g.close()

Using the above we could already manipulated setting files, or any files with some algorithm we choose. For the read and replace it is interesting to use line by line processing with `fileinput`

In [ ]:
import fileinput

In [ ]:
for line in fileinput.input('burza'):
    if 'ARIEL' in line:
        print(line)
    if line.startswith('KALIBAN'):
        print(line)

In [ ]:
print(line)
a = line.split()
print(a)
b = ' '.join(a)
print(b)

```
    for line in fileinput.input(mshfile, inplace=1):
        if line.startswith("S="):
            print ("S="+str(S)+";")
        elif line.startswith("a="):
            print "a="+str(alfa)+";"
        else:
            print line
```

**Coding Time!**: Use `fileinput` with `inplace` to modify the file. For example change names of selected characters. `Ariel->Perwol` `Kaliban->Taliban` etc.

## Matplotlib
is a very popular plotting library for python. It can be used to produce almost publication quality figures. Personally I use it for most of my plotting needs, but for publications I prefer Latex Tix library.

Let us start with our first plot of some `f(x)` function. We will be using `nympy.array` to handle our data. To use `matplotlib` we need to import it:

### First plot

In [ ]:
import matplotlib.pyplot as plt

To have a plot for $x \in [-1, 1]$ we will use `numpy.linspace()` we skipped last time:

In [ ]:
import numpy as np

x = np.linspace(-1, 1, 101)
# Generate 101 points between -1 and 1 including boundaries
y = x**2

And plotting `y(x)` is just:

In [ ]:
plt.plot(x, y)

We can do many things with the plot. Add grid, set axis limits, add text, labels, change sizes, colors, etc.

For now let just increase the size of the plot, add a legend entry  a grid, change the font size and thickness of the line.

We can also do a scatter plot:

In [ ]:
f = plt.figure(figsize=(20,12)) # figure size
font = {'family' : 'Comic Sans MS', 'weight' : 'normal', 'size'   : 24} #larger font I like
plt.rc('font', **font)

plt.plot(x, y, label='$f(x)$', lw=10)
plt.scatter(0,0, s=500, c='black')

plt.grid()
plt.legend()
plt.xlim(x[0], x[-1])
plt.ylim(min(y)-0.05, max(y)+0.05)

So we can make a plot:
* Add log scaling
* Show plotting with subplots
* TBA

In [ ]:
fig, axs = plt.subplots(2,2)
axs[0,0].plot(x, y)
axs[0,0].grid()
axs[0,1].plot(x, -y)
axs[1,0].plot(x, y)
axs[1,1].plot(x, -y)

## Reading from files (with data)
We consider a computational process monitoring of which is performed with a text data stored to a file during the calculation. Here I have a file `mean.avg` that contains average values of some vector field in time:

In [ ]:
ls

Let's start by examining the content of the file (in notepad).

The file contains columns of numbers (possibly comma separated, or not) so the best way to treat is is to precess it directly to NymPy arrays using NumPy functions. Note: Processing of ``noraml`` text files can also be performed, but in case of data files using NumPy functions is more natural.

In [ ]:
T, c1, c2, c3 = np.loadtxt('mean.avg', comments="#", skiprows=1, usecols=(0,1,2,3), unpack=True)

In [ ]:
plt.plot(T, c1, T, c2, T, c3)
plt.grid()

**Coding Time!**:
This one is a bigger one. A small visualization project, but graphs only! Use data in `data.tar.gz` to plot history of flow rate of the perturbation in time. The file structure is such: ...

## SciPy
Is the second probably most commonly (my opinion rather than fact) used library when it comes to processing data. You could say, that when it comes to numerical procedures - SciPy has it all. We will focus on only some of the available algorithms, but should you need more - you know where to look.

We will:
* Decimate data with slice operations
* Use interpolation to build a continuous view of data
* Look for extrema
* Look for roots
* Integrate
* Solve an ODE
* Curve fitting

and all that in a day work.

We will use a slice of data from mean.avg, using the slice operation with collection\[start : stop : every n'th\]. The last entry is used to select only some of the data.

In [ ]:
X =  T[10000:13000:250] # We can select every other value from an array!
Y = c3[10000:13000:250]
print(len(X), len(Y))

Lets plot the original data, and the points selected from it and stored in `X` and `Y`:

In [ ]:
plt.plot(T, c3)
plt.scatter(X, Y)

plt.xlim(X[0]-100, X[-1]+100)
plt.grid()

### Interpolation
There is a number of interpolation procedures available from Scipy, the most up to date list can be found here: https://docs.scipy.org/doc/scipy/reference/interpolate.html  

Choosing the right interpolation technique is vary much problem dependant, might require some trial-and-error  and here we will focus on the use of only some more general SciPy interpolation procedures, allowing to easily switch between interpolation techniques.

**Note:** We will only do 1D interpolation, but higher dimensions are also possible, see e.g. [CloughTocher2DInterpolator](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.CloughTocher2DInterpolator.html#scipy.interpolate.CloughTocher2DInterpolator).

#### `scipy.interpolate.interp1d`
The first is the `interp1D`, it allows for different types of **interpolation**, but does not allow for **extrapolation**. The advantage is that it is very easy to use. A general call to `interp1D` needs the pass of arrays of values used to build the interpolating function of a selected kind. The `kind` can be any of the implemented. The default is `'linear'`, but we will look at: `zero`, `slinear`, `quadratic` and `cubic`  - feturing [splines](https://en.wikipedia.org/wiki/Spline_%28mathematics%29) of a respective order:

**Note** the difference between *interpolation*, which is the act of approximating a value in between the known values and *extrapolation*, which is guessing outside of the known values.

To use `interp1D` lets import it from `scipy` module:

In [ ]:
from scipy.interpolate import interp1d

`interp1d` returns a function object to be used with data:

In [ ]:
f = interp1d(X, Y) # default is liner

Which we can use as any other function with Python, with a single variable, a list or a NumPy array:

In [ ]:
print(f(8000))
print(f(X[3]))
print(f(X[3:6]))

Lets, see how good our interpolation is, using linear distribution of points:

In [ ]:
x = np.linspace(X[0], X[-1], 10000) # only interpolation
# x = np.linspace(X[0]-100, X[-1]+100, 1000) # interp1d does not work with extrapolation!

**Note:** Show the pitfalls of dynamic typing.  
Replace `fig = plt.figure(figsize=(20,12))`  
with `f = plt.figure(figsize=(20,12))` which you normally use and have a nice error!

In [ ]:
fig = plt.figure(figsize=(20,12)) # figure size
font = {'family' : 'Comic Sans MS', 'weight' : 'normal', 'size'   : 24} #larger font I like
plt.rc('font', **font)

plt.plot(T, c3, '--', c='black') # '--' - dashed, c - colour, 
plt.scatter(X, Y, s=100) # s sets size
plt.plot(x, f(x), lw=5) # lw - line width

plt.xlim(X[0]-100, X[-1]+100)
plt.ylim(0.275, 0.3)
plt.grid()

And we have a nice linear interpolation. Let's try the remaining types, and get a glimpse at how interpolating functions differ:

In [ ]:
fzero    = interp1d(X, Y, kind='zero') # default is liner
fslinear = interp1d(X, Y, kind='slinear')
fquadratic = interp1d(X, Y, kind='quadratic')
fcubic = interp1d(X, Y, kind='cubic')

We will take this occasion to add a legend to our plot. This is done by placing a `label='label text'` in the `plot()` call and calling on the `legend()` function.

**Note:** `label=''` accepts Latex mathematical environment with \$\$. 

In [ ]:
fig = plt.figure(figsize=(20,12)) # figure size
font = {'family' : 'Comic Sans MS', 'weight' : 'normal', 'size'   : 24} #larger font I like
plt.rc('font', **font)

plt.plot(T, c3, '--', c='black') # '--' - dashed, c - colour, 
plt.scatter(X, Y, s=100) # s sets size

# Note the use of label in the function call - we will have a legend!
plt.plot(x, f(x), lw=5, label='linear') # 
plt.plot(x, fzero(x), lw=5, label='zero') # 
plt.plot(x, fslinear(x), lw=5, label='slinear') # 
plt.plot(x, fquadratic(x), lw=5, label='quadratic') # 
plt.plot(x, fcubic(x), lw=5, label='cubic $\gamma_{sub}^{sup}(x)$') # $ math goes here $

# It is going to be Legendary!
plt.legend()
plt.xlim(X[0]-100, X[-1]+100)
plt.ylim(0.275, 0.3)
plt.grid()

#### `scipy.interpolate.InterpolatedUnivariateSpline`
Returns a 1D interpolating spline for a provided data. Aside from data we may pass weights, degree of the spline (`$1<=k<=5$`) and a flag specifyingwhat should happen out of data bounds, the extrapolation `ext $\in (0,3)$`.

In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline

x = np.linspace(X[0]-100, X[-1]+100, 1000) # we will be extrapolating!

f = InterpolatedUnivariateSpline(X, Y, k=1)

We will use this moment to show how to look for array indices that correspond to a given value. For the case of a sorted array it is probably best to use the `numpy.searchsorted` function. It accepts a sorted collection and values (or a value) and returns an index where the value should be inserted to maintain the order (sortiness??) of the collection. Example:

In [ ]:
a = np.searchsorted(x, X[0])
b = np.searchsorted(x, X[-1])
print(a, b)

print(x[a], X[0])
print(x[b], X[-1])

`a` and `b` are indices of the first greater than `X[0]` and `X[-1]` elements of `x`.

In [ ]:
fig = plt.figure(figsize=(20,12)) # figure size
font = {'family' : 'Comic Sans MS', 'weight' : 'normal', 'size'   : 24} #larger font I like
plt.rc('font', **font)

plt.plot(T, c3, '--', c='black') # '--' - dashed, c - colour, 
plt.scatter(X, Y, s=100) # s sets size

a = np.searchsorted(x, X[0])
b = np.searchsorted(x, X[-1])
p = plt.plot(x[a:b], f(x[a:b]), lw=5) # note, we return to p
plt.plot(x[0:a], f(x[0:a]), '--', lw=5, c = p[0].get_color()) # to get the same colour
plt.plot(x[b:-1], f(x[b:-1]), '--', lw=5, c = p[0].get_color())

ax = plt.gca()
ax.annotate('The extrapolated part!', xy=(x[25], f(x[25])), xytext=(7600, 0.288),
            arrowprops = dict(facecolor ='black', width=8, headwidth=25, headlength=25, shrink = 0.05),)

plt.xlim(X[0]-150, X[-1]+150)
plt.ylim(0.275, 0.3)
plt.grid()

Note the use of `annotate` to place an arrow and some text to our plot.

Lets experiment with `exp` and `k`:
* `ext=0` or `‘extrapolate’` - default.
* `ext=1` or `‘zeros’`, returns 0.
* `ext=2` or `‘raise’`, raise a `ValueError`.
* `ext=3` of `‘const’`, return boundary value.

In [ ]:
f = InterpolatedUnivariateSpline(X, Y, k=1, ext=3)

fig = plt.figure(figsize=(20,12)) # figure size
font = {'family' : 'Comic Sans MS', 'weight' : 'normal', 'size'   : 24} #larger font I like
plt.rc('font', **font)

plt.plot(T, c3, '--', c='black') # '--' - dashed, c - colour, 
plt.scatter(X, Y, s=100) # s sets size

a = np.searchsorted(x, X[0])
b = np.searchsorted(x, X[-1])
p = plt.plot(x[a:b], f(x[a:b]), lw=5) # note, we return to p
plt.plot(x[0:a], f(x[0:a]), '--', lw=5, c = p[0].get_color()) # to get the same colour
plt.plot(x[b:-1], f(x[b:-1]), '--', lw=5, c = p[0].get_color())

ax = plt.gca()
ax.annotate('The boundary value!', xy=(x[25], f(x[25])), xytext=(7600, 0.288),
            arrowprops = dict(facecolor ='black', width=8, headwidth=25, headlength=25, shrink = 0.05),)

plt.xlim(X[0]-150, X[-1]+150)
plt.ylim(0.275, 0.3)
plt.grid()

In [ ]:
f1 = InterpolatedUnivariateSpline(X, Y, k=1)
f2 = InterpolatedUnivariateSpline(X, Y, k=2)
f3 = InterpolatedUnivariateSpline(X, Y, k=3)
f4 = InterpolatedUnivariateSpline(X, Y, k=4)
f5 = InterpolatedUnivariateSpline(X, Y, k=5)

In [ ]:
fig = plt.figure(figsize=(20,12)) # figure size
font = {'family' : 'Comic Sans MS', 'weight' : 'normal', 'size'   : 24} #larger font I like
plt.rc('font', **font)

plt.plot(T, c3, '--', c='black') # '--' - dashed, c - colour, 
plt.scatter(X, Y, s=100) # s sets size

plt.plot(x, f1(x), lw=5, label='1')
plt.plot(x, f2(x), lw=5, label='2')
plt.plot(x, f3(x), lw=5, label='3')
plt.plot(x, f4(x), lw=5, label='4')
plt.plot(x, f5(x), lw=5, label='5')

plt.legend()
plt.xlim(X[0]-150, X[-1]+150)
plt.ylim(0.275, 0.3)
plt.grid()

### Finding extreme value
We know how to build an interpolating function from data. Let's say the points we have result from some form of a process we monitor and that we would like to find the extreme value. We will assume the process to be represented by some interpolant and look for the maximum. Note we are not interested in the maximum, stored in the array (`max(f(x))`), but in the extrema of a function, that corresponds to whatever our problem is.

Let start with just examining the arrays:

In [ ]:
print( max(Y), max( f2(x)) )
i = np.where(f2(x) == max(f2(x)))
print(i, x[i], f2(x[i]))

So an element of `x` corresponds to a value returned by an interpolating function `f2` (one of many) that happens to be the elemental maximum - nice, but not what we want yet.

In what fallows we will use a `scipy.optimize` module minimization procedure `scipy.optimize.minimize`. As before should you need something more fancy, have a look at the documentation [here](https://docs.scipy.org/doc/scipy/reference/optimize.html). I will use the `f2` created from interpolation before.

We need to import an appropriate module:

In [ ]:
from scipy.optimize import minimize

`minimize` takes a number of arguments. Two are required the function to be minimized and the initial guess. You can also specify the particular method you wish to use (see docs), bounds, tolerance, etc.

The returned value is an object containing a number of informations. To us the most important is the result.   
**(experiment with starting value, and bounds)**

In [ ]:
# res = minimize(f2, 8450)
res = minimize(f2, x0=8600, bounds=((8400, 8800),) ) # note the nasty way bounds are provided
print(res)

In [ ]:
fig = plt.figure(figsize=(20,12)) # figure size
font = {'family' : 'Comic Sans MS', 'weight' : 'normal', 'size'   : 24} #larger font I like
plt.rc('font', **font)

plt.plot(T, c3, '--', c='black') # '--' - dashed, c - colour, 
plt.scatter(X, Y, s=100) # s sets size

plt.plot(x, f2(x), lw=5, label='2')
plt.scatter(res.x, f2(res.x), s=200)

plt.legend()
plt.xlim(X[0]-150, X[-1]+150)
plt.ylim(0.275, 0.3)
plt.grid()

**Note:** Should we want to find the maximum, a small addition is needed:

In [ ]:
def f22(x):
    return -1*f2(x)

In [ ]:
res = minimize(f22, x0=0.5*(x[0]+X[-1]), bounds=((X[0], X[-1]),) )
print(res)

In [ ]:
fig = plt.figure(figsize=(20,12)) # figure size
font = {'family' : 'Comic Sans MS', 'weight' : 'normal', 'size'   : 24} #larger font I like
plt.rc('font', **font)

plt.plot(T, c3, '--', c='black') # '--' - dashed, c - colour, 
plt.scatter(X, Y, s=100) # s sets size

plt.plot(x, f2(x), lw=5, label='2')
plt.scatter(res.x, f2(res.x), s=200)

plt.legend()
plt.xlim(X[0]-150, X[-1]+150)
plt.ylim(0.275, 0.3)
plt.grid()

#### Example:
We will now write a piece of code to improve localization of an minimum. Starting with some `X` and `Y` arrays that mimic a limited numbers of test that we have on the actual process represented by `x` and `y` we will iterate by finding minima of an interpolating functions, than choosing those as points to probe `x` and `y` and inserting values data arrays used for interpolation:

In [ ]:
X =  T[10000:14000:1000] # We can select every other value from an array!
Y = c3[10000:14000:1000]
x = np.linspace(X[0], X[-1], 10000)
print(len(X), len(Y))

-> iteration starts here <-

In [ ]:
f2 = InterpolatedUnivariateSpline(X, Y, k=2)
res = minimize(f2, x0=8600, bounds=((X[0], X[-1]),), tol=1e-12)
print(res)
print(res.x[0], f2(res.x[0]))

In [ ]:
fig = plt.figure(figsize=(20,12)) # figure size
font = {'family' : 'Comic Sans MS', 'weight' : 'normal', 'size'   : 24} #larger font I like
plt.rc('font', **font)

plt.plot(T, c3, '--', c='black') # '--' - dashed, c - colour, 
plt.scatter(X, Y, s=100) # s sets size
plt.plot(x, f2(x))
plt.scatter(res.x, f2(res.x), s=500)

plt.xlim(X[0]-150, X[-1]+150)
plt.ylim(0.275, 0.3)
plt.grid()

We now need to run our process for the value of an argument we determined to result in the minimum of the interpolating function (if it is the best strategy, that is another story). In our case this is equivalent to finding the values in `T` and `c3` arrays:

In [ ]:
idx = np.searchsorted(T, res.x)
newX = T[idx]
newY = c3[idx]
print(idx, res.x, newX, newY)

We will use `numpy.insert` to update our `X` and `Y` with newly acquired values. `insert` needs to now the index before which it should insert:

In [ ]:
a = np.searchsorted(X, newX)
X = np.insert(X, a, newX)
Y = np.insert(Y, a, newY)

Redo the interpolation:

In [ ]:
f2 = InterpolatedUnivariateSpline(X, Y, k=2)

In [ ]:
fig = plt.figure(figsize=(20,12)) # figure size
font = {'family' : 'Comic Sans MS', 'weight' : 'normal', 'size'   : 24} #larger font I like
plt.rc('font', **font)

plt.plot(T, c3, '--', c='black') # '--' - dashed, c - colour, 
plt.scatter(X, Y, s=100) # s sets size
plt.plot(x, f2(x))
plt.scatter(res.x, f2(res.x), s=500)

plt.xlim(X[0]-150, X[-1]+150)
plt.ylim(0.275, 0.3)
plt.grid()

We can reiterate, or close everything in a nice loop and a procedure.

### Roots
As with the others there is a number of procedures, and it is up to you to select the one that is right for your problem.

In [ ]:
from scipy.optimize import root_scalar

In [ ]:
def fun(x):
    return f2(x) - 0.280

In [ ]:
res = root_scalar(fun, x0=8400, x1=8800)

In [ ]:
print(res, res.root, fun(res.root))

### Curve fitting
Here we show an example of curve fitting using least-square approximation. We try to find parameters of a function that best fits some points:

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def fun(x, a, b, c):
    return a*x**5 + b*x**3 + c

In [ ]:
popt, pcov = curve_fit(fun, X, Y)
print(popt)

In [ ]:
fig = plt.figure(figsize=(20,12)) # figure size
font = {'family' : 'Comic Sans MS', 'weight' : 'normal', 'size'   : 24} #larger font I like
plt.rc('font', **font)

plt.plot(T, c3, '--', c='black') # '--' - dashed, c - colour, 
plt.scatter(X, Y, s=100) # s sets size
plt.plot(x, fun(x, *popt))

plt.legend()
plt.xlim(X[0]-150, X[-1]+150)
plt.ylim(0.275, 0.3)
plt.grid()

Not the best fit, but well, works and for some real data might be actually the thing.

### Integration
SciPy provides a number of integration techniques:

In [ ]:
from scipy.integrate import quad # general purpose integration/

In [ ]:
res = quad(f2, a=X[0], b=X[-1])
print(res[0])

### Solving a simple ODE
via `from scipy.integrate.ode` an ODE system of the form $x'(t)=f(t,x)$ might be solved.

In [ ]:
from scipy.integrate import odeint

`ODE` needs the right hand side function defining derivative `f` for a given state `x` and time `t`.

Let consider a system [Lorenz System](https://en.wikipedia.org/wiki/Lorenz_system) (kind of famous - see why), for the evolution of a vector of variables ${\bf x}=[x,y,z]$ in time $t$, according to:  
$$
\begin{cases}
\frac{dx}{dt} = \sigma(y - x)\\
\frac{dy}{dt} = x(\rho - z) - y\\
\frac{dz}{dt} = xy - \beta z
\end{cases}
$$

With initial conditions ${\bf x}(t=0) = [x_0, y_0, z_0]$.

The function defining the derivative is simply:

In [ ]:
sigma=10
beta=8./3
rho=28.0

def lor_rhs(x0, t0):
    return [sigma * (x0[1] - x0[0]), x0[0] * (rho - x0[2]) - x0[1], x0[0] * x0[1] - beta * x0[2]]

In [ ]:
x0 = [1, 1, 1]
t = np.linspace(0, 20, 10000)  # one thousand time steps
xt = odeint(lor_rhs, x0, t)

We will now plot. New thing is the use of `%matplotlib notebook` that allows us to produce an interactive plot:

In [ ]:
%matplotlib notebook

from matplotlib import pyplot as plt    # this one we know
from mpl_toolkits.mplot3d import Axes3D # Produces 3D axes setting

fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1], projection='3d')

x, y, z = xt.T
ax.plot3D(x, y, z)

# prepare the axes limits
ax.set_xlim((-25, 25))
ax.set_ylim((-35, 35))
ax.set_zlim((5, 55))

# set point-of-view: specified by (altitude degrees, azimuth degrees)
ax.view_init(30, 0)

In [ ]:
N = 20
# Choose random starting points, uniformly distributed from -15 to 15
np.random.seed(1)
x0 = -15 + 30 * np.random.random((N, 3))

# Solve for the trajectories
t = np.linspace(0, 20, 10000)
xt = np.asarray([integrate.odeint(lor_rhs, x0i, t)
                  for x0i in x0])


fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1], projection='3d')

for xx in xt:
    x, y, z = xx.T
    ax.plot3D(x, y, z)

# prepare the axes limits
ax.set_xlim((-50, 50))
ax.set_ylim((-50, 50))
ax.set_zlim((5, 55))

# set point-of-view: specified by (altitude degrees, azimuth degrees)
ax.view_init(30, 0)